In [6]:
import astropy.units as u

from generate_priors_TOI import grab_all_priors
from RV_mini_pipeline import remake_rvfile
from generate_supporting_files import *
import sys
sys.path.insert(0, '/Users/jack/Research/pipelines/exofast_tools/scripts/lightcurve_pipeline/')
from lightcurve_pipeline import create_light_curve
import pandas as pd
import lightkurve as lk

## System parameters

In [2]:
toi = '3601'
ticid = '191202679'
rv_instrument = 'tres'
outpath = f'/Users/jack/Research/meep3/fits/toi{toi}/'

## Generating priors and SED
Before running this cell, create a directory to store the fit files and generate the prior and SED files using mkticsed

In [3]:
with open('tres_password.txt', 'r') as file:
    password = file.read().replace('\n', '')

grab_all_priors(f'TOI-{toi}', outpath=outpath, source='tres', tres_username='schulte', tres_password=password)

Starting points:
Tc = 2459879.372447
Period = 4.0517972
Rp/Rs = 0.07909487973314075
Spectroscopic [Fe/H] prior: 0.2385 +/- 0.1073
Dilution prior: 0.0 +/- 0.0013418


## Downloading and formatting RVs
Ensure that the units and output filename are correct for the given instrument. Usually, CHIRON RVs are in km/s and TRES RVs are in m/s

In [4]:
path = outpath + f'toi{toi}.TRES.rv'

with open('tres_password.txt', 'r') as file:
    password = file.read().replace('\n', '')

remake_rvfile(path, u.m/u.s, output_filename=outpath+f'{ticid}.TRES.rv', download_rvs=True, username='schulte', password=password, ticid=ticid, instrument=rv_instrument, verbose=True)

Processed file:
       Time (BJD)  RV (m/s)  RV error (m/s)
0   2.459461e+06     720.8            52.6
1   2.459471e+06     242.6            82.5
2   2.460203e+06     668.4            51.3
3   2.460205e+06     123.8            47.0
4   2.460208e+06     150.8            42.3
5   2.460209e+06      37.5            59.4
6   2.460212e+06     332.6            44.7
7   2.460213e+06      -0.2            43.5
8   2.460217e+06     -49.4            40.3
9   2.460219e+06     710.8            60.4
10  2.460220e+06     175.8            50.3
11  2.460222e+06     332.1            53.0
12  2.460227e+06     557.8            36.8
13  2.460228e+06     254.9            50.7


## Downloading TESS lightcurves
Run the first cell to see what lightcurves are available, and then modify and run the next cell to download the selected lightcurve. Make sure that you are downloading the highest cadence lightcurves available and that the data are of good quality.

After running the initial EXOFAST fit, you can check for a secondary eclipse by re-running these cells with the correct parameters.

In [ ]:
lk.search_lightcurve(f'TIC {ticid}')

In [ ]:
create_light_curve(f'TIC {ticid}', 'SPOC', 65, auto = True, exposure=120, targetname = f'TOI-{toi}', keep_secondary_eclipse=False,
                   ecosw = -0.541, t14s = None, save = True, plot = True, planet='b', published = False, qualityFlag = False, #omit_transit_index=[4],
                   outputfiles='slimflat')

## Generating scripts to run the fit

After collecting all lightcurves, change the exptimes array in the following cell and then run it to generate the final files.

In [2]:
outpath = f'/Users/jack/Research/meep3/fits/toi{toi}/'
exptimes = [30, 30, 10, 10, 10, 2, 2]
generate_pro_file(toi, ticid, exptimes, outpath=outpath)
generate_SLURM_file(toi, outpath=outpath)